In [1]:
# Dependencies
import pandas as pd
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time
from prettyprinter import pprint
import json

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\brook\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
store_states = ['dc', 'de', 'md', 'va']

In [4]:
cities_list = []

for store_state in store_states:
    
    url_state = 'https://stores.giantfood.com/' + store_state
    browser.visit(url_state)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    resultsX = soup.find_all('li', class_="DirectoryList-item")
    
    store_cities = resultsX
    
    
    for store_city in store_cities:

        try:
            city = store_city.find('a', class_="DirectoryList-itemLink Link--secondary").text        

            city_dict = {            
                'city': city,
                'state': store_state
            }

            cities_list.append(city_dict)

        except Exception as e:
            print(e)
            pass    

In [5]:
cities_df = pd.DataFrame.from_dict(cities_list)
cities_df['url'] = 'https://stores.giantfood.com/' + cities_df['state'] + '/' + cities_df['city'].str.lower().str.replace(' ', '-')
cities_df.head()

,city,state,url
0,Washington,dc,https://stores.giantfood.com/dc/washington
1,Bear,de,https://stores.giantfood.com/de/bear
2,Middletown,de,https://stores.giantfood.com/de/middletown
3,Millsboro,de,https://stores.giantfood.com/de/millsboro
4,Millville,de,https://stores.giantfood.com/de/millville


In [6]:
city_urls = cities_df.url

In [7]:
stores_list = []

for city_url in city_urls:
    
    url = city_url
    browser.visit(url)
    
    time.sleep(1)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    results = soup.find_all('li', class_="LocationList-item l-col-xs-12 l-col-sm-6 l-col-md-3-up")
    
    stores = results
    
    for store in stores:

        try:
            street_address = store.find('span', class_='c-address-street-1').text
            city = store.find('span', class_='c-address-city').text
            state = store.find('abbr', class_='c-address-state').text
            zip_code = store.find('span', class_='c-address-postal-code').text

            product_dict = {
                'street_address': street_address,
                'city': city,
                'state': state,                
                'zip_code': zip_code
            }

            stores_list.append(product_dict)

        except Exception as e:
            print(e)
            pass

In [8]:
stores_df = pd.DataFrame.from_dict(stores_list)
stores_df

,street_address,city,state,zip_code
0,1050 Brentwood Road,Washington,DC,20018
1,"1345 Park Road, NW",Washington,DC,20010
2,1400 7th Street NW,Washington,DC,20001
3,1535 Alabama Avenue SE,Washington,DC,20032
4,300 H Street NE,Washington,DC,20002
...,...,...,...,...
159,317 Worth Avenue,Stafford,VA,22554
160,21000 Southbank Street,Sterling,VA,20165
161,21800 Town Center Plaza,Sterling,VA,20164
162,359 Maple Avenue East,Vienna,VA,22180
